In [1]:
from artem import *

In [2]:
def task(m, n):
    transform  = get_transform(r_prim[m], q_prim[n])
    
    q_avg_tree = KDTree(apply_transform(q_avg, transform))
    
    dist = r_avg_tree.sparse_distance_matrix(
        q_avg_tree,
        matchrange,
        p=2,
        output_type='ndarray'
    )
    
    nb   = mutual_nb(dist)
    
    size = len(nb)
    if not sizemin <= size <= sizemax:
        return None
    
    scnd = vstack([[r_scnd[i], q_scnd[j]] for i, j in nb])
    transform  = get_transform(*scnd)
    
    r_coord, q_coord = vstack([[r_eval[i], q_eval[j]] for i, j in nb])
    q_coord_tran = apply_transform(q_coord, transform)
    
    rmsd = RMSD(r_coord, q_coord_tran)
    if not rmsdmin <= rmsd <= rmsdmax:
        return None
    
    rmsdsize = rmsd / size
    if not rmsdsizemin <= rmsdsize <= rmsdsizemax:
        return None
    
    nb = tuple(sorted(nb))
    return [size, rmsd, rmsdsize, nb, transform]

In [3]:
argv = [
    'r=data/1xjr.cif', 
    'q=data/2gis.pdb', 
    # 'rmsdmax=3', 
    # 'sizemin=20',
    # 'saveto=results', 
    'threads=-1',
    # 'rres=/X:A_0_100',
    # 'rresneg=/X:_100_500',
    # 'rseed=/X:A_0_20' 
]

In [4]:
kwargs = dict([arg.split('=') for arg in argv])

threads = int(kwargs.get('threads', threads))
if threads != 1:
    # cur_sys = system()
    # Multiprocessing is available only for UNIX-like systems
    # if cur_sys in {'Linux', 'Darwin'}:
    mp.set_start_method('fork')
    if threads < 0:
        threads = mp.cpu_count()
    else:
        threads = min(threads, mp.cpu_count())


r       = kwargs.get('r')
rres    = kwargs.get('rres', rres)
rresneg = kwargs.get('rresneg', rresneg)
rseed   = kwargs.get('rseed', rseed)
rformat = kwargs.get('rformat', rformat)

q       = kwargs.get('q')
qres    = kwargs.get('qres', qres)
qresneg = kwargs.get('qresneg', qresneg)
qseed   = kwargs.get('qseed', qseed)
qformat = kwargs.get('qformat', qformat)

sizemin     = float(kwargs.get('sizemin', sizemin))
sizemax     = float(kwargs.get('sizemax', sizemax))

rmsdmin     = float(kwargs.get('rmsdmin', rmsdmin))
rmsdmax     = float(kwargs.get('rmsdmax', rmsdmax))

rmsdsizemin = float(kwargs.get('rmsdsizemin', rmsdsizemin))
rmsdsizemax = float(kwargs.get('rmsdsizemax', rmsdsizemax))

matchrange  = float(kwargs.get('matchrange', matchrange))

saveto     = kwargs.get('saveto', saveto)
saveres    = kwargs.get('saveres', saveres)

rname, rext = r.split(os.sep)[-1].split('.')
qname, qext = q.split(os.sep)[-1].split('.')

available_format = {'PDB', 'CIF'}

rext = rext.upper()
if rext in available_format:
    rformat = rext

qext = qext.upper()
if qext in available_format:
    qformat = qext

saveformat = kwargs.get('saveformat', qformat)

In [5]:
rseed = ':_19G'

In [6]:
rstruct  = pdb.parser(r, rformat, rname)
rstruct.drop_duplicates_alt_id(keep=keep)
rnegcase = bool(rresneg)
rsstruct = rstruct.get_res_substruct(
    [rres, rresneg][rnegcase],
    rnegcase
)
rrres, rures = rsstruct.artem_desc(seed_res_repr)
rseed_code = rsstruct.get_res_code(rseed)

r_code, r_prim, r_avg, r_scnd, r_eval = zip(*rrres)
r_avg = np.vstack(r_avg)
rseed_code = set(r_code) & set(rseed_code)
r_ind = [i for i, code in enumerate(r_code) if code in rseed_code]

In [22]:
rseed = '#/A:A_19_30'

In [23]:
rsstruct.get_res_code(rseed)

['1.A.A.23.', '1.A.A.26.', '1.A.A.29.']

In [8]:
'sfd.fsdf.dsf'.islower()

True

In [16]:
rsstruct.tab[rsstruct.tab['auth_seq_id']==19]

,group_PDB,id,type_symbol,label_atom_id,label_alt_id,label_comp_id,label_asym_id,label_entity_id,label_seq_id,pdbx_PDB_ins_code,...,Cartn_y,Cartn_z,occupancy,B_iso_or_equiv,pdbx_formal_charge,auth_seq_id,auth_comp_id,auth_asym_id,auth_atom_id,pdbx_PDB_model_num
395,ATOM,396,P,P,.,G,A,1,19,?,...,34.206,46.580,1.0,70.28,?,19,G,A,P,1
396,ATOM,397,O,OP1,.,G,A,1,19,?,...,33.435,46.073,1.0,67.12,?,19,G,A,OP1,1
397,ATOM,398,O,OP2,.,G,A,1,19,?,...,35.587,46.099,1.0,68.35,?,19,G,A,OP2,1
398,ATOM,399,O,O5',.,G,A,1,19,?,...,34.303,48.175,1.0,70.48,?,19,G,A,O5',1
399,ATOM,400,C,C5',.,G,A,1,19,?,...,33.167,49.040,1.0,71.77,?,19,G,A,C5',1
400,ATOM,401,C,C4',.,G,A,1,19,?,...,33.604,50.467,1.0,72.35,?,19,G,A,C4',1
401,ATOM,402,O,O4',.,G,A,1,19,?,...,34.153,50.464,1.0,72.52,?,19,G,A,O4',1
402,ATOM,403,C,C3',.,G,A,1,19,?,...,34.675,51.113,1.0,70.38,?,19,G,A,C3',1
403,ATOM,404,O,O3',.,G,A,1,19,?,...,34.540,52.532,1.0,70.33,?,19,G,A,O3',1
404,ATOM,405,C,C2',.,G,A,1,19,?,...,35.927,50.681,1.0,70.07,?,19,G,A,C2',1


In [13]:
rsstruct.get_res_code(rseed)

[]

In [6]:
qstruct  = pdb.parser(q, qformat, qname)
qstruct.drop_duplicates_alt_id(keep=keep)
qnegcase = bool(qresneg)
qsstruct = qstruct.get_res_substruct(
    [qres, qresneg],
    qnegcase
)
qrres, qures = qsstruct.artem_desc(seed_res_repr)
qseed_code = qsstruct.get_res_code(qseed)

q_code, q_prim, q_avg, q_scnd, q_eval = zip(*qrres)
q_avg = np.vstack(q_avg)
qseed_code = set(q_code) & set(qseed_code)
q_ind = [i for i, code in enumerate(q_code) if code in qseed_code]

In [7]:
indx_pairs = list(itertools.product(r_ind, q_ind))
r_avg_tree = KDTree(r_avg)

In [13]:
if threads == 1:
    result = [task(m, n) for m, n in indx_pairs]
else:
    pool = mp.Pool(threads)
    
    delta   = 15 * threads
    cnt     = 0
    cnt_max = len(indx_pairs)
    result  = []
    
    while cnt < cnt_max:
        result.extend(
            pool.starmap(task, indx_pairs[cnt:cnt + delta])
        )
        cnt += delta

In [15]:
items = {}
for i, item in enumerate(result):
    if item:
        nb = item[-2]
        if nb not in items:
            items[nb] = item + [[i]]
        else:
            items[nb][-1].append(i)
            
            # selecting the minimum RMSD
            if item[1] < items[nb][1]:
                items[nb][1] = item[1]
                items[nb][2] = item[2]

In [33]:
columns = ['SIZE', 'RMSD', 'RMSDSIZE', 'PRIM', 'SCND', 'TRAN']
tab     = []
for code in rures:
    tab.append([0, None, None, rsstruct.name, code, None])
for code in qures:
    tab.append([0, None, None, qsstruct.name, code, None])

In [34]:
for item in items.values():
    seeds = item[-1]
    seeds_code = []
    for ind in seeds:
        i, j = indx_pairs[ind]
        seeds_code.append('='.join([r_code[i], q_code[j]]))
    seeds_code = ','.join(seeds_code)
    
    match_code = []
    for i, j in item[-3]:
        match_code.append('='.join([r_code[i], q_code[j]]))
    match_code = ','.join(match_code)
    tab.append([*item[:3], seeds_code, match_code, item[4]])

In [37]:
tab = pd.DataFrame(tab, columns=columns)
tab.sort_values(
    ['SIZE', 'RMSDSIZE'], 
    ascending=[True, False], 
    inplace=True
)

In [38]:
tab

,SIZE,RMSD,RMSDSIZE,PRIM,SCND,TRAN
0,0,NaN,NaN,1xjr,1.A.GTP.1.,None
1,0,NaN,NaN,1xjr,1.A.MG.101.,None
2,0,NaN,NaN,1xjr,1.A.MG.201.,None
3,0,NaN,NaN,1xjr,1.A.HOH.202.,None
4,0,NaN,NaN,1xjr,1.A.HOH.203.,None
...,...,...,...,...,...,...
119,24,2.236159e+00,9.317327e-02,1.A.G.4.=1.A.A.3.,"1.A.A.3.=1.A.G.2.,1.A.G.4.=1.A.A.3.,1.A.U.5.=1...","([[0.9890203532854964, 0.13895820524947358, 0...."
75,24,2.236159e+00,9.317327e-02,1.A.A.3.=1.A.G.4.,"1.A.G.2.=1.A.A.3.,1.A.A.3.=1.A.G.4.,1.A.G.4.=1...","([[0.9890203532854964, -0.1021502761494541, -0..."
1986,26,2.346592e+00,9.025353e-02,1.A.U.47.=1.A.U.46.,"1.A.G.2.=1.A.A.3.,1.A.A.3.=1.A.G.4.,1.A.G.4.=1...","([[0.9928138473201883, -0.11863261523468831, -..."
1949,26,2.346592e+00,9.025353e-02,1.A.U.46.=1.A.U.47.,"1.A.A.3.=1.A.G.2.,1.A.G.4.=1.A.A.3.,1.A.U.5.=1...","([[0.9928138473201877, 0.11557364466059396, -0..."


In [4]:
'ds1f43.'.islower()

True

In [6]:
'.'.join('1')

'1'

In [7]:
'dfsdds.f'.isalnum()

False